In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import tensorflow as tf
from pandas import read_csv
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils

from sklearn.metrics import mean_squared_error, r2_score

from pandas import read_csv
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras import optimizers

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import chart_studio.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)

import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

import json

In [2]:
df2 = pd.read_csv("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/real_pm10_1.csv")
df2.head()

/home/intern/parkjh/anaconda3/envs/datastory/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning:

Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.



,Unnamed: 0,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
0,0,2019,12,5,2019-12-05,15:53,5348,2019-12-05 15:53,37.47957,126.87713,7.60,8.67,15
1,1,2019,12,5,2019-12-05,15:55,5348,2019-12-05 15:55,37.47957,126.87713,7.17,7.77,15
2,2,2019,12,5,2019-12-05,15:57,5348,2019-12-05 15:57,37.47957,126.87713,8.17,9.00,15
3,3,2019,12,5,2019-12-05,15:59,5348,2019-12-05 15:59,37.47957,126.87713,8.01,8.62,15
4,4,2019,12,5,2019-12-05,16:01,5348,2019-12-05 16:01,37.47957,126.87713,7.94,8.65,16


1912LYS134 시간별, 일별 추이

In [3]:
df_134 = df2[df2.EQPMN_ID == '1912LYS134']

In [4]:
df_134

,Unnamed: 0,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
27581,27581,2019,12,18,2019-12-18,17:30,1912LYS134,2019-12-18 17:30,36.9143,127.0626,12.0,13.0,17
27582,27582,2019,12,18,2019-12-18,17:40,1912LYS134,2019-12-18 17:40,36.9143,127.0626,18.0,20.0,17
27583,27583,2019,12,18,2019-12-18,17:50,1912LYS134,2019-12-18 17:50,36.9143,127.0626,14.0,16.0,17
27584,27584,2019,12,18,2019-12-18,18:00,1912LYS134,2019-12-18 18:00,36.9143,127.0626,14.0,18.0,18
27585,27585,2019,12,18,2019-12-18,18:10,1912LYS134,2019-12-18 18:10,36.9143,127.0626,14.0,17.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782614,1782614,2020,9,30,2020-09-30,23:10:12,1912LYS134,2020-09-30 23:10:12,36.9143,127.0626,13.0,14.0,23
1782615,1782615,2020,9,30,2020-09-30,23:20:13,1912LYS134,2020-09-30 23:20:13,36.9143,127.0626,13.0,14.0,23
1782616,1782616,2020,9,30,2020-09-30,23:30:14,1912LYS134,2020-09-30 23:30:14,36.9143,127.0626,13.0,14.0,23
1782617,1782617,2020,9,30,2020-09-30,23:40:12,1912LYS134,2020-09-30 23:40:12,36.9143,127.0626,13.0,14.0,23


In [5]:
for i, time in enumerate(['hour', 'day']):

    lable = ["시간별", "일별"]
    date = ["시간대", "일"]
    PTC_VAL_134_time = df_134.groupby(time).mean()['PTC_VAL']

    img_1 = px.bar(PTC_VAL_134_time, 
                       x=PTC_VAL_134_time.index, 
                       y="PTC_VAL", 
                       color='PTC_VAL',
                   color_continuous_scale=px.colors.sequential.Bluered
                  )

    img_1.update_layout(
        xaxis = dict(
            tickfont = dict(family="Noto Sans KR"),
            tickvals = PTC_VAL_134_time.index
        ),
        xaxis_title=date[i],
        yaxis_title="미세먼지 수치 (PTC_VAL)",
        title={
            'text': "1912LYS134 %s 미세먼지 추이" %(lable[i]),
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            font=dict(
                family="Noto Sans KR",
                size=14
            ),
        showlegend=False,
        coloraxis_showscale = False
    )
    
    # png
    img_1.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_1_%d.png" %(i+1))
    
    #json
    img_1_json = img_1.to_json()
    img_1_json_obj = json.loads(img_1_json, encoding="utf-8")

    with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_1_%d.json' %(i+1), 'w', encoding='utf-8') as make_file:
        json.dump(img_1_json_obj, make_file, indent="\t", ensure_ascii=False)

    img_1.show()
    

1912LYS141 시간별, 일별 추이

In [6]:
df_141 = df2[df2.EQPMN_ID == '1912LYS141']
df_141

,Unnamed: 0,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
40958,40958,2019,12,18,2019-12-18,17:30,1912LYS141,2019-12-18 17:30,36.9058,126.693,10.0,31.0,17
40959,40959,2019,12,18,2019-12-18,17:40,1912LYS141,2019-12-18 17:40,36.9058,126.693,11.0,36.0,17
40960,40960,2019,12,18,2019-12-18,17:50,1912LYS141,2019-12-18 17:50,36.9058,126.693,10.0,30.0,17
40961,40961,2019,12,18,2019-12-18,18:00,1912LYS141,2019-12-18 18:00,36.9058,126.693,11.0,34.0,18
40962,40962,2019,12,18,2019-12-18,18:10,1912LYS141,2019-12-18 18:10,36.9058,126.693,11.0,32.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800743,1800743,2020,9,30,2020-09-30,23:10:12,1912LYS141,2020-09-30 23:10:12,36.9058,126.693,7.0,15.0,23
1800744,1800744,2020,9,30,2020-09-30,23:20:13,1912LYS141,2020-09-30 23:20:13,36.9058,126.693,7.0,17.0,23
1800745,1800745,2020,9,30,2020-09-30,23:30:14,1912LYS141,2020-09-30 23:30:14,36.9058,126.693,6.0,16.0,23
1800746,1800746,2020,9,30,2020-09-30,23:40:12,1912LYS141,2020-09-30 23:40:12,36.9058,126.693,5.0,13.0,23


In [7]:
for i, time in enumerate(['hour', 'day']):

    lable = ["시간별", "일별"]
    date = ["시간대", "일"]
    PTC_VAL_141_time = df_141.groupby(time).mean()['PTC_VAL']

    img_2 = px.bar(PTC_VAL_141_time, 
                       x=PTC_VAL_141_time.index, 
                       y="PTC_VAL", 
                       color='PTC_VAL',
                   color_continuous_scale=px.colors.sequential.Bluered
                  )

    img_2.update_layout(
        xaxis = dict(
            tickfont = dict(family="Noto Sans KR"),
            tickvals = PTC_VAL_141_time.index
        ),
        xaxis_title=date[i],
        yaxis_title="미세먼지 수치 (PTC_VAL)",
        title={
            'text': "1912LYS141 %s 미세먼지 추이" %(lable[i]),
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            font=dict(
                family="Noto Sans KR",
                size=14
            ),
        showlegend=False,
        coloraxis_showscale = False
    )

    # png
    img_2.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_2_%d.png" %(i+1))
    
    #json
    img_2_json = img_2.to_json()
    img_2_json_obj = json.loads(img_2_json, encoding="utf-8")

    with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_2_%d.json' %(i+1), 'w', encoding='utf-8') as make_file:
        json.dump(img_2_json_obj, make_file, indent="\t", ensure_ascii=False)
    
    img_2.show()
    

서울 중구 미세먼지 추이 (시간, 계절)

In [8]:
df3 = pd.read_csv("/home/datastory/datastory_mod/실외 미세먼지 데이터를 통한 미세먼지 예측/pm10_2001.csv")
df3.columns

Index(['Unnamed: 0', 'dates', 'year', 'month', 'day', 'time', 'PM10', 'PM2.5',
       'O3', 'NO2', 'CO', 'SO2'],
      dtype='object')

In [9]:
pm10_junggu_time = df3.groupby('time').mean()['PM10']

In [10]:
img_3_1 = px.bar(pm10_junggu_time, 
                   x=pm10_junggu_time.index, 
                   y="PM10", 
                   color='PM10',
               color_continuous_scale=px.colors.sequential.Bluered
              )

img_3_1.update_layout(
    xaxis = dict(
        tickfont = dict(family="Noto Sans KR"),
        tickvals = PTC_VAL_141_time.index
    ),
    xaxis_title="시간대",
    yaxis_title="미세먼지 수치 (PM10)",
    title={
        'text': "서울 중구 시간별 미세먼지 추이",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(
            family="Noto Sans KR",
            size=14
        ),
    showlegend=False,
    coloraxis_showscale = False
)

# png
img_3_1.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_3_1.png")

#json
img_3_1_json = img_3_1.to_json()
img_3_1_json_obj = json.loads(img_3_1_json, encoding="utf-8")

with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_3_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(img_3_1_json_obj, make_file, indent="\t", ensure_ascii=False)


img_3_1.show()


서울 중구 계절별 미세먼지 추이 

In [11]:
seasons = df3['PM10'].groupby(df3['month']).mean()

In [12]:
season = pd.DataFrame(seasons)
season

,PM10
month,
1,59.560811
2,52.682635
3,62.273954
4,35.340782
5,43.578019
6,23.048815
7,26.165312
8,24.431786
9,20.416079


In [13]:
winter = (season['PM10'][1] + season['PM10'][2] + season['PM10'][12]) / 3
spring = (season['PM10'][3] + season['PM10'][4] + season['PM10'][5]) / 3
summmer = (season['PM10'][6] + season['PM10'][7] + season['PM10'][8]) / 3 
fall = (season['PM10'][9] + season['PM10'][10] + season['PM10'][11]) / 3 

season_list = [winter, spring, summmer, fall]

In [14]:
season_table = pd.DataFrame(season_list)

In [15]:
label_season = ['겨울','봄','여름','가을']

season_table.index = label_season
season_table

,0
겨울,50.851700
봄,47.064252
여름,24.548637
가을,27.835067


In [16]:
img_3_2 = px.bar(season_table, 
                   x=season_table.index, 
                   y=season_table[0], 
                   color=season_table[0],
               color_continuous_scale=px.colors.sequential.Bluered
              )

img_3_2.update_layout(
    xaxis_title="계절",
    yaxis_title="미세먼지 수치 (PM10)",
    title={
        'text': "서울 중구 계절별 미세먼지 추이",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(
            family="Noto Sans KR",
            size=14
        ),
    showlegend=False,
    coloraxis_showscale = False
)

# png
img_3_2.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_3_2.png")

#json
img_3_2_json = img_3_2.to_json()
img_3_2_json_obj = json.loads(img_3_2_json, encoding="utf-8")

with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_3_2.json', 'w', encoding='utf-8') as make_file:
    json.dump(img_3_2_json_obj, make_file, indent="\t", ensure_ascii=False)


img_3_2.show()

    

서울 중구 미세먼지 추이 (일별, 월별)

In [17]:
for i, time in enumerate(['day', 'month']):

    lable = ["일별", "월별"]
    date = ["일", "월"]
    pm10_junggu_time = df3.groupby(time).mean()['PM10']

    img_4 = px.bar(pm10_junggu_time, 
                       x=pm10_junggu_time.index, 
                       y="PM10", 
                       color='PM10',
                   color_continuous_scale=px.colors.sequential.Bluered
                  )

    img_4.update_layout(
        xaxis = dict(
            tickfont = dict(family="Noto Sans KR"),
            tickvals = pm10_junggu_time.index
        ),
        xaxis_title=date[i],
        yaxis_title="미세먼지 수치 (PM10)",
        title={
            'text': "서울시 중구 2019년 %s 미세먼지 추이" %(lable[i]),
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            font=dict(
                family="Noto Sans KR",
                size=14
            ),
        showlegend=False,
        coloraxis_showscale = False
    )

    # png
    img_4.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_4_%d.png" %(i+1))
    
    #json
    img_4_json = img_4.to_json()
    img_4_json_obj = json.loads(img_4_json, encoding="utf-8")

    with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_4_%d.json' %(i+1), 'w', encoding='utf-8') as make_file:
        json.dump(img_4_json_obj, make_file, indent="\t", ensure_ascii=False)
    
    img_4.show()
    

성분별 상관관계

In [18]:
df_air_condition = df3[['PM10', 'PM2.5', 'O3', 'NO2', 'CO', 'SO2']] 
df_air_condition

,PM10,PM2.5,O3,NO2,CO,SO2
0,39.0,24.0,0.003,0.054,0.7,0.003
1,26.0,16.0,0.030,0.017,0.4,0.003
2,31.0,21.0,0.022,0.026,0.4,0.003
3,32.0,21.0,0.024,0.027,0.4,0.004
4,32.0,23.0,0.031,0.021,0.4,0.003
...,...,...,...,...,...,...
10074,36.0,23.0,0.026,0.008,0.4,0.003
10075,36.0,21.0,0.027,0.007,0.4,0.003
10076,59.0,23.0,0.026,0.009,0.4,0.002
10077,21.0,16.0,0.028,0.008,0.3,0.003


In [19]:
df_corr = df_air_condition.corr()

df_corr.columns = ['미세먼지', '초미세먼지', '오존', '이산화질소', '일산화탄소', '아황산가스']
df_corr.index = df_corr.columns

z = df_corr.values[::-1]
x = df_corr.columns
y = df_corr.index[::-1]

annotations = []
for n,row in enumerate(z):
    for m, val in enumerate(row):
        annotations.append(
            dict(text='{0:.2f}'.format(z[n][m]) if not np.isnan(z[n][m]) else '',
                x=x[m],
                y=y[n],
                xref='x1',
                yref='y1',
                showarrow=False,
                font=dict(
                    family="Noto Sans KR",
                    size=14,
                color='#fff')))
        
trace = go.Heatmap(x=x, y=y, z=z, colorscale='turbo')
img_5_1 = go.Figure(data=trace)

img_5_1.update_layout(
    annotations=annotations,
    #title_text="7일 데이터",
    #title_x=0.5,
    font=dict(
        family="Noto Sans KR",
        size=14),
    title={
            'text': "상관 분석",
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
)


# png
img_5_1.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_5_1.png" )

#json
img_5_1_json = img_5_1.to_json()
img_5_1_json_obj = json.loads(img_5_1_json, encoding="utf-8")

with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_5_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(img_5_1_json_obj, make_file, indent="\t", ensure_ascii=False)

img_5_1.show()


## 1912LYS134, 1912LYS141,에어코리아의 예측치, 실측치 비교



### 1. 1912LYS134

* 전처리

In [20]:
df = pd.read_csv('real_pm10_1.csv', encoding = 'utf-8-sig',
                                    skiprows=1,
           names = ['year','month','day','date','time','EQPMN_ID','DATA_OCCRRNC_DT','LA','LO','ULFPTC_VAL','PTC_VAL','hour'])

/home/intern/parkjh/anaconda3/envs/datastory/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning:

Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.



In [21]:
b = df[df.EQPMN_ID == '1912LYS134']

In [22]:
b.reset_index(inplace=True)

In [23]:
df = b.copy()

In [24]:
b = b.replace([np.inf,-np.inf], np.nan).dropna(subset=['ULFPTC_VAL','PTC_VAL'], how='all')
b = b.dropna(subset=['PTC_VAL'])
b = b.dropna(subset=['ULFPTC_VAL'])
b

,index,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
0,27581,2019,12,18,2019-12-18,17:30,1912LYS134,2019-12-18 17:30,36.9143,127.0626,12.0,13.0,17
1,27582,2019,12,18,2019-12-18,17:40,1912LYS134,2019-12-18 17:40,36.9143,127.0626,18.0,20.0,17
2,27583,2019,12,18,2019-12-18,17:50,1912LYS134,2019-12-18 17:50,36.9143,127.0626,14.0,16.0,17
3,27584,2019,12,18,2019-12-18,18:00,1912LYS134,2019-12-18 18:00,36.9143,127.0626,14.0,18.0,18
4,27585,2019,12,18,2019-12-18,18:10,1912LYS134,2019-12-18 18:10,36.9143,127.0626,14.0,17.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39159,1782614,2020,9,30,2020-09-30,23:10:12,1912LYS134,2020-09-30 23:10:12,36.9143,127.0626,13.0,14.0,23
39160,1782615,2020,9,30,2020-09-30,23:20:13,1912LYS134,2020-09-30 23:20:13,36.9143,127.0626,13.0,14.0,23
39161,1782616,2020,9,30,2020-09-30,23:30:14,1912LYS134,2020-09-30 23:30:14,36.9143,127.0626,13.0,14.0,23
39162,1782617,2020,9,30,2020-09-30,23:40:12,1912LYS134,2020-09-30 23:40:12,36.9143,127.0626,13.0,14.0,23


In [25]:
df.index = pd.to_datetime(list(df['date']))
df

,index,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
2019-12-18,27581,2019,12,18,2019-12-18,17:30,1912LYS134,2019-12-18 17:30,36.9143,127.0626,12.0,13.0,17
2019-12-18,27582,2019,12,18,2019-12-18,17:40,1912LYS134,2019-12-18 17:40,36.9143,127.0626,18.0,20.0,17
2019-12-18,27583,2019,12,18,2019-12-18,17:50,1912LYS134,2019-12-18 17:50,36.9143,127.0626,14.0,16.0,17
2019-12-18,27584,2019,12,18,2019-12-18,18:00,1912LYS134,2019-12-18 18:00,36.9143,127.0626,14.0,18.0,18
2019-12-18,27585,2019,12,18,2019-12-18,18:10,1912LYS134,2019-12-18 18:10,36.9143,127.0626,14.0,17.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-30,1782614,2020,9,30,2020-09-30,23:10:12,1912LYS134,2020-09-30 23:10:12,36.9143,127.0626,13.0,14.0,23
2020-09-30,1782615,2020,9,30,2020-09-30,23:20:13,1912LYS134,2020-09-30 23:20:13,36.9143,127.0626,13.0,14.0,23
2020-09-30,1782616,2020,9,30,2020-09-30,23:30:14,1912LYS134,2020-09-30 23:30:14,36.9143,127.0626,13.0,14.0,23
2020-09-30,1782617,2020,9,30,2020-09-30,23:40:12,1912LYS134,2020-09-30 23:40:12,36.9143,127.0626,13.0,14.0,23


* Split

In [26]:
start_date = df.index[-1] - pd.tseries.offsets.MonthBegin(1)

In [27]:
train_set_size = len(df) - len(df[start_date:])
train_set_size

36574

In [28]:
raw_train = b[:train_set_size]
raw_test = b[train_set_size:]

In [29]:
train_cols = ['ULFPTC_VAL','PTC_VAL']
test_cols = ['PTC_VAL']

In [30]:
# train 셋, test 셋
x_train = raw_train.loc[:, train_cols].values
y_train = raw_train.loc[:, test_cols].values

x_test = raw_test.loc[:, train_cols].values
y_test = raw_test.loc[:, test_cols].values

In [31]:
#정규화
min_max_scaler = MinMaxScaler()
min_max_scaler_label = MinMaxScaler()

x_train_scaled = min_max_scaler.fit_transform(x_train)
y_train_scaled = min_max_scaler_label.fit_transform(y_train)

x_test_scaled = min_max_scaler.transform(x_test)
y_test_scaled = min_max_scaler_label.transform(y_test)

In [32]:
# 데이터를 시계열로 변환
def build_timeseries(mat, time_steps):
    dim_0 = mat.shape[0] - time_steps #데이터의 길이(레코드 수) - time_steps
    dim_1 = mat.shape[1] #컬럼 수 
    
    x = np.zeros((dim_0, time_steps, dim_1)) #dim_0개의 time_steps x dim_1 인 0행렬을 생성
    y = np.zeros((dim_0,)) #길이가 dim_0이고 값은 0인 배열 생성

    for i in range(dim_0):
        x[i] = mat[i:time_steps + i] # 데이터 셋을0~0+time_steps, 1~1+time_steps ... 로 분할
        y[i] = mat[time_steps + i, 0] # 데이터 셋의 배열 0번째 컬럼 값 (수온)
        
    print("length of time-series i/o", x.shape, y.shape)
    return x, y

In [33]:
x_train_scaled_time, y_train_scaled_time = build_timeseries(x_train_scaled, 1)
x_test_scaled_time, y_test_scaled_time = build_timeseries(x_test_scaled, 1)

length of time-series i/o (36573, 1, 2) (36573,)
length of time-series i/o (2589, 1, 2) (2589,)


In [34]:
from keras.layers import Dense, Activation, Flatten

In [35]:
def create_model(input_data):
    lstm_model = Sequential()
    lstm_model.add(Bidirectional(LSTM(100, return_sequences = True),  input_shape=(x_train_scaled_time.shape[1], x_train_scaled_time.shape[2])))
    lstm_model.add(Dense(4))
    lstm_model.add(Flatten())
    lstm_model.add(Dense(1))
    lstm_model.compile(loss='mse', optimizer='adam')
    
    return lstm_model

In [36]:
lstm_model = create_model(x_train_scaled_time)

In [37]:
lstm_model.summary() #모델 요약 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1, 200)            82400     
_________________________________________________________________
dense (Dense)                (None, 1, 4)              804       
_________________________________________________________________
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 83,209
Trainable params: 83,209
Non-trainable params: 0
_________________________________________________________________


In [38]:
x_train_scaled_time.shape

(36573, 1, 2)

In [39]:
y_train_scaled_time

array([0.1682243 , 0.13084112, 0.13084112, ..., 0.00934579, 0.00934579,
       0.00934579])

In [40]:
history_134 = lstm_model.fit(x_train_scaled_time, 
                         y_train_scaled_time, epochs=30, 
                         verbose=1, 
                         batch_size=16, 
                         shuffle=False, 
                         validation_split=0.2)

Epoch 1/30
1829/1829 [==============================] - 4s 1ms/step - loss: 0.0020 - val_loss: 8.9441e-05
Epoch 2/30
1829/1829 [==============================] - 2s 898us/step - loss: 5.9467e-04 - val_loss: 6.8730e-05
Epoch 3/30
1829/1829 [==============================] - 2s 864us/step - loss: 5.7172e-04 - val_loss: 6.3535e-05
Epoch 4/30
1829/1829 [==============================] - 2s 854us/step - loss: 5.5639e-04 - val_loss: 6.7290e-05
Epoch 5/30
1829/1829 [==============================] - 2s 861us/step - loss: 5.4783e-04 - val_loss: 7.5134e-05
Epoch 6/30
1829/1829 [==============================] - 2s 854us/step - loss: 5.4163e-04 - val_loss: 8.3120e-05
Epoch 7/30
1829/1829 [==============================] - 2s 857us/step - loss: 5.3553e-04 - val_loss: 8.9502e-05
Epoch 8/30
1829/1829 [==============================] - 2s 856us/step - loss: 5.3058e-04 - val_loss: 9.3634e-05
Epoch 9/30
1829/1829 [==============================] - 2s 869us/step - loss: 5.2665e-04 - val_loss: 9.6761e-0

In [41]:
y_predicted = lstm_model.predict(x_test_scaled_time) # 모델의 출력 예측 
y_predicted_value = min_max_scaler_label.inverse_transform(y_predicted) #본래 값으로 돌리기
y_test_value = min_max_scaler_label.inverse_transform(y_test_scaled_time.reshape(-1, 1)) #

In [42]:
# 실제 데이터와 예측 데이터 비교하는 데이터프레임
compare_data_134 = pd.DataFrame()
compare_data_134['실측치'] = y_test_value.flatten()
compare_data_134['예측치'] = np.round(y_predicted_value,1)

In [43]:
compare_data_134

,실측치,예측치
0,1.233645,1.900000
1,1.233645,1.600000
2,1.233645,1.600000
3,1.233645,1.600000
4,1.233645,1.600000
...,...,...
2584,16.037383,16.700001
2585,16.037383,16.700001
2586,16.037383,16.700001
2587,16.037383,16.700001


### 2. 1912LYS141

* 전처리


In [44]:
df = pd.read_csv('real_pm10_1.csv', encoding = 'utf-8-sig',
                                    skiprows=1,
           names = ['year','month','day','date','time','EQPMN_ID','DATA_OCCRRNC_DT','LA','LO','ULFPTC_VAL','PTC_VAL','hour'])

/home/intern/parkjh/anaconda3/envs/datastory/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning:

Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.



In [45]:
c = df[df.EQPMN_ID == '1912LYS141']
c.reset_index(inplace=True)

In [46]:
df = c.copy()

In [47]:
c = c.replace([np.inf,-np.inf], np.nan).dropna(subset=['ULFPTC_VAL','PTC_VAL'], how='all')
c = c.dropna(subset=['PTC_VAL'])
c = c.dropna(subset=['ULFPTC_VAL'])
c

,index,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
0,40958,2019,12,18,2019-12-18,17:30,1912LYS141,2019-12-18 17:30,36.9058,126.693,10.0,31.0,17
1,40959,2019,12,18,2019-12-18,17:40,1912LYS141,2019-12-18 17:40,36.9058,126.693,11.0,36.0,17
2,40960,2019,12,18,2019-12-18,17:50,1912LYS141,2019-12-18 17:50,36.9058,126.693,10.0,30.0,17
3,40961,2019,12,18,2019-12-18,18:00,1912LYS141,2019-12-18 18:00,36.9058,126.693,11.0,34.0,18
4,40962,2019,12,18,2019-12-18,18:10,1912LYS141,2019-12-18 18:10,36.9058,126.693,11.0,32.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39163,1800743,2020,9,30,2020-09-30,23:10:12,1912LYS141,2020-09-30 23:10:12,36.9058,126.693,7.0,15.0,23
39164,1800744,2020,9,30,2020-09-30,23:20:13,1912LYS141,2020-09-30 23:20:13,36.9058,126.693,7.0,17.0,23
39165,1800745,2020,9,30,2020-09-30,23:30:14,1912LYS141,2020-09-30 23:30:14,36.9058,126.693,6.0,16.0,23
39166,1800746,2020,9,30,2020-09-30,23:40:12,1912LYS141,2020-09-30 23:40:12,36.9058,126.693,5.0,13.0,23


In [48]:
df.index = pd.to_datetime(list(df['date']))
df

,index,year,month,day,date,time,EQPMN_ID,DATA_OCCRRNC_DT,LA,LO,ULFPTC_VAL,PTC_VAL,hour
2019-12-18,40958,2019,12,18,2019-12-18,17:30,1912LYS141,2019-12-18 17:30,36.9058,126.693,10.0,31.0,17
2019-12-18,40959,2019,12,18,2019-12-18,17:40,1912LYS141,2019-12-18 17:40,36.9058,126.693,11.0,36.0,17
2019-12-18,40960,2019,12,18,2019-12-18,17:50,1912LYS141,2019-12-18 17:50,36.9058,126.693,10.0,30.0,17
2019-12-18,40961,2019,12,18,2019-12-18,18:00,1912LYS141,2019-12-18 18:00,36.9058,126.693,11.0,34.0,18
2019-12-18,40962,2019,12,18,2019-12-18,18:10,1912LYS141,2019-12-18 18:10,36.9058,126.693,11.0,32.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-30,1800743,2020,9,30,2020-09-30,23:10:12,1912LYS141,2020-09-30 23:10:12,36.9058,126.693,7.0,15.0,23
2020-09-30,1800744,2020,9,30,2020-09-30,23:20:13,1912LYS141,2020-09-30 23:20:13,36.9058,126.693,7.0,17.0,23
2020-09-30,1800745,2020,9,30,2020-09-30,23:30:14,1912LYS141,2020-09-30 23:30:14,36.9058,126.693,6.0,16.0,23
2020-09-30,1800746,2020,9,30,2020-09-30,23:40:12,1912LYS141,2020-09-30 23:40:12,36.9058,126.693,5.0,13.0,23


* Split 

In [49]:
start_date = df.index[-1] - pd.tseries.offsets.MonthBegin(1)

In [50]:
train_set_size = len(df) - len(df[start_date:])

In [51]:
train_set_size

36578

In [52]:
raw_train = c[:train_set_size]
raw_test = c[train_set_size:]

In [53]:
train_cols = ['ULFPTC_VAL','PTC_VAL']
test_cols = ['PTC_VAL']

In [54]:
# train 셋, test 셋
x_train = raw_train.loc[:, train_cols].values
y_train = raw_train.loc[:, test_cols].values

x_test = raw_test.loc[:, train_cols].values
y_test = raw_test.loc[:, test_cols].values

In [55]:
#정규화
min_max_scaler = MinMaxScaler()
min_max_scaler_label = MinMaxScaler()

x_train_scaled = min_max_scaler.fit_transform(x_train)
y_train_scaled = min_max_scaler_label.fit_transform(y_train)

x_test_scaled = min_max_scaler.transform(x_test)
y_test_scaled = min_max_scaler_label.transform(y_test)

In [56]:
x_train_scaled_time, y_train_scaled_time = build_timeseries(x_train_scaled, 1)
x_test_scaled_time, y_test_scaled_time = build_timeseries(x_test_scaled, 1)

length of time-series i/o (36577, 1, 2) (36577,)
length of time-series i/o (2589, 1, 2) (2589,)


In [57]:
from keras.layers import Dense, Activation, Flatten

In [58]:
lstm_model = create_model(x_train_scaled_time)

In [59]:
lstm_model.summary() #모델 요약 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1, 200)            82400     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 4)              804       
_________________________________________________________________
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 83,209
Trainable params: 83,209
Non-trainable params: 0
_________________________________________________________________


In [60]:
x_train_scaled_time.shape

(36577, 1, 2)

In [61]:
y_train_scaled_time

array([0.10091743, 0.09174312, 0.10091743, ..., 0.39449541, 0.39449541,
       0.3853211 ])

In [62]:
history_141 = lstm_model.fit(x_train_scaled_time, 
                         y_train_scaled_time, epochs=30, 
                         verbose=1, 
                         batch_size=16, 
                         shuffle=False, 
                         validation_split=0.2)

Epoch 1/30
1829/1829 [==============================] - 3s 1ms/step - loss: 0.0020 - val_loss: 0.0015
Epoch 2/30
1829/1829 [==============================] - 2s 853us/step - loss: 0.0011 - val_loss: 4.0657e-04
Epoch 3/30
1829/1829 [==============================] - 2s 846us/step - loss: 0.0011 - val_loss: 2.0905e-04
Epoch 4/30
1829/1829 [==============================] - 2s 858us/step - loss: 0.0010 - val_loss: 1.6627e-04
Epoch 5/30
1829/1829 [==============================] - 2s 846us/step - loss: 0.0010 - val_loss: 1.5969e-04
Epoch 6/30
1829/1829 [==============================] - 2s 851us/step - loss: 0.0010 - val_loss: 1.5850e-04
Epoch 7/30
1829/1829 [==============================] - 2s 850us/step - loss: 9.9242e-04 - val_loss: 1.6119e-04
Epoch 8/30
1829/1829 [==============================] - 2s 848us/step - loss: 9.8519e-04 - val_loss: 1.6087e-04
Epoch 9/30
1829/1829 [==============================] - 2s 846us/step - loss: 9.7698e-04 - val_loss: 1.6207e-04
Epoch 10/30
1829/1829 

In [63]:
y_predicted = lstm_model.predict(x_test_scaled_time) # 모델의 출력 예측 
y_predicted_value = min_max_scaler_label.inverse_transform(y_predicted) #본래 값으로 돌리기
y_test_value = min_max_scaler_label.inverse_transform(y_test_scaled_time.reshape(-1, 1)) #

In [64]:
# 실제 데이터와 예측 데이터 비교하는 데이터프레임
compare_data_141 = pd.DataFrame()
compare_data_141['실측치'] = y_test_value.flatten()
compare_data_141['예측치'] = np.round(y_predicted_value,1)

In [65]:
compare_data_141

,실측치,예측치
0,347.229358,336.899994
1,356.366972,329.799988
2,374.642202,337.200012
3,365.504587,351.500000
4,365.504587,344.700012
...,...,...
2584,63.963303,56.000000
2585,63.963303,65.000000
2586,54.825688,65.099998
2587,45.688073,56.200001


## 3. 에어코리아

In [66]:
raw_train = pd.read_csv('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/에어코리아_2021/pm10_2019.csv', 
                        encoding = 'utf-8-sig',
                        skiprows=1,
                        names = ['dates','year','month','day','time','PM10','PM2.5','O3','NO2','CO','SO2'])

In [67]:
raw_test = df = pd.read_csv("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/air_korea_202001.csv", 
                            encoding = 'utf-8-sig',
                            skiprows=1,
                            names = ['dates','year','month','day','time','PM10','PM2.5','O3','NO2','CO','SO2'])

In [68]:
train_cols = ['PM10','PM2.5','O3','NO2','CO','SO2']
test_cols = ['PM10']

# train 셋, test 셋
x_train = raw_train.loc[:, train_cols].values
y_train = raw_train.loc[:, test_cols].values

x_test = raw_test.loc[:, train_cols].values
y_test = raw_test.loc[:, test_cols].values


#정규화
min_max_scaler = MinMaxScaler()
min_max_scaler_label = MinMaxScaler()

x_train_scaled = min_max_scaler.fit_transform(x_train)
y_train_scaled = min_max_scaler_label.fit_transform(y_train)

x_test_scaled = min_max_scaler.transform(x_test)
y_test_scaled = min_max_scaler_label.transform(y_test)

In [69]:
x_train_scaled_time, y_train_scaled_time = build_timeseries(x_train_scaled, 24)
x_test_scaled_time, y_test_scaled_time = build_timeseries(x_test_scaled, 24)

length of time-series i/o (8647, 24, 6) (8647,)
length of time-series i/o (709, 24, 6) (709,)


In [70]:
def create_model_air(input_data):
    lstm_model = Sequential()
    lstm_model.add(Bidirectional(LSTM(6, return_sequences = True),  input_shape=(x_train_scaled_time.shape[1], x_train_scaled_time.shape[2])))
    lstm_model.add(Flatten())
    lstm_model.add(Dense(1))
    lstm_model.compile(loss='mse', optimizer='adam')

    return lstm_model

In [71]:
lstm_model = create_model_air(x_train_scaled_time)
lstm_model.summary() #모델 요약 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 24, 12)            624       
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 289       
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


In [72]:
history_air = lstm_model.fit(x_train_scaled_time, y_train_scaled_time, 
                         epochs=10, verbose=1, batch_size=16, shuffle=False, validation_split=0.2)

Epoch 1/10
433/433 [==============================] - 3s 4ms/step - loss: 0.0183 - val_loss: 0.0078
Epoch 2/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0086 - val_loss: 0.0062
Epoch 3/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0064 - val_loss: 0.0058
Epoch 4/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0057 - val_loss: 0.0056
Epoch 5/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0054 - val_loss: 0.0054
Epoch 6/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0053 - val_loss: 0.0053
Epoch 7/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0052 - val_loss: 0.0052
Epoch 8/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0051 - val_loss: 0.0052
Epoch 9/10
433/433 [==============================] - 2s 3ms/step - loss: 0.0050 - val_loss: 0.0051
Epoch 10/10
433/433 [==============================] - 1s 3ms/step - loss: 0.0050 - val_loss: 0.0051

In [73]:
y_predicted = lstm_model.predict(x_test_scaled_time) # 모델의 출력 예측 
y_predicted_value = min_max_scaler_label.inverse_transform(y_predicted) #본래 값으로 돌리기
y_test_value = min_max_scaler_label.inverse_transform(y_test_scaled_time.reshape(-1, 1)) #

In [74]:
# 실제 데이터와 예측 데이터 비교하는 데이터프레임
compare_data_air = pd.DataFrame()
compare_data_air['실측치'] = y_test_value.flatten()
compare_data_air['예측치'] = np.round(y_predicted_value,1)

In [75]:
compare_data_air

,실측치,예측치
0,63.0,29.500000
1,75.0,44.200001
2,74.0,60.700001
3,71.0,69.400002
4,73.0,72.199997
...,...,...
704,8.0,12.800000
705,9.0,13.600000
706,9.0,13.400000
707,19.0,14.300000


### 1912LYS134, 1912LYS141, 에어코리아의 예측치 및 실측치 비교

In [76]:
compare_data = [compare_data_134, compare_data_141, compare_data_air]
label = ['1912LYS134', '1912LYS141', '에어코리아']
unit = ['PTC_VAL','PTC_VAL', 'PM10']

In [77]:
for i, data in enumerate(compare_data):

    img_6 = go.Figure()
    img_6.add_trace(go.Scatter(x=data.index, y=data['예측치'],
                        mode='lines',
                        name='예측치',
                        marker_color='navy'))
    img_6.add_trace(go.Scatter(x=data.index, y=data['실측치'],
                        mode='lines',
                        name='실측치',
                        marker_color='orange'))
    
    unit = ['PTC_VAL','PTC_VAL', 'PM10']
    img_6.update_layout(
        xaxis_title="경과일수",
        yaxis_title="미세먼지 수치 (%s)" %(unit[i]),
        font=dict(
            family="Noto Sans KR",
            size=14),
        title={
                'text': "%s의 예측치, 실측치 비교" %(label[i]),
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'}
    )

    # png
    img_6.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_6_%d.png" %(i+1) )

    #json
    img_6_json = img_6.to_json()
    img_6_obj = json.loads(img_6_json, encoding="utf-8")

    with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_6_%d.json' %(i+1), 'w', encoding='utf-8') as make_file:
        json.dump(img_6_obj, make_file, indent="\t", ensure_ascii=False)
        
    img_6.show()


### 검증, 손실 그래프


* 1912LYS134, 1912LYS141, 에어코리아

In [78]:
history_list = [history_134, history_141, history_air]

for i, list in enumerate(history_list):

    img_7 = go.Figure()

    img_7.add_trace(go.Scatter(y=list.history['loss'],
                        mode='lines',
                        name='훈련 데이터',
                        marker_color='navy'))

    img_7.add_trace(go.Scatter(y=list.history['val_loss'],
                        mode='lines',
                        name='검증 데이터',
                        marker_color='orange'))
    

    img_7.update_layout(
        xaxis_title="훈련 횟수",
        yaxis_title="손실 값",
        yaxis=dict(tickformat='.4f'),
        font=dict(
            family="Noto Sans KR",
            size=14),
        title={
                'text': "%s의 손실 값 비교" %(label[i]),
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'}
        )


    # png
    img_7.write_image("/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_7_%d.png" %(i+1))

    #json
    img_7_json = img_7.to_json()
    img_7_json_obj = json.loads(img_7_json, encoding="utf-8")

    with open('/home/datastory/10.실외 미세먼지 데이터를 통한 미세먼지 예측/10/img_7_%d.json' %(i+1), 'w', encoding='utf-8') as make_file:
        json.dump(img_7_json_obj, make_file, indent="\t", ensure_ascii=False)


    img_7.show()
    
    

In [79]:
#mape
def mean_absolute_percentage_error(y_test, y_pred):
    y_true, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [80]:
#rmse
def rmse_r(y, y_pred):
    return mean_squared_error(y, y_pred) ** 0.5

In [81]:
mape = mean_absolute_percentage_error(y_test_value, y_predicted_value)
rmse = rmse_r(y_test_value, y_predicted_value)

In [82]:
print("mape :", mape)
print("rsme :", rmse)

mape : 25.09831667892354
rsme : 10.660579251363014
